In [1]:
from __future__ import absolute_import
from __future__ import division

import os
import sys
import math
import random
import shutil
import argparse
import importlib
import data_utils
import numpy as np
import pointfly as pf
import tensorflow as tf
from datetime import datetime


/usr/local/lib/python3.5/dist-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
args = argparse.Namespace(
    setting = "amsterdam_x4_12288_fps",
    filelist = "../data/Amsterdam/train_data_files.txt",
    filelist_val = "../data/Amsterdam/filelists/val_files_g_3.txt",
    save_folder = "../models/seg/",
    model = "pointcnn_seg",
    load_ckpt = '../models/seg/pointcnn_seg_amsterdam_x4_12288_fps_2019-02-05-22-40-26_20662/ckpts/iter-25173',
    
#     load_ckpt = None,
    epochs = None,
    batch_size = None,
    log = "-",
    no_timestamp_folder = None,
    no_code_backup = None
)

In [3]:
# parser = argparse.ArgumentParser()
# parser.add_argument('--filelist', '-t', help='Path to training set ground truth (.txt)', required=True)
# parser.add_argument('--filelist_val', '-v', help='Path to validation set ground truth (.txt)', required=True)
# parser.add_argument('--load_ckpt', '-l', help='Path to a check point file for load')
# parser.add_argument('--save_folder', '-s', help='Path to folder for saving check points and summary', required=True)
# parser.add_argument('--model', '-m', help='Model to use', required=True)
# parser.add_argument('--setting', '-x', help='Setting to use', required=True)
# parser.add_argument('--epochs', help='Number of training epochs (default defined in setting)', type=int)
# parser.add_argument('--batch_size', help='Batch size (default defined in setting)', type=int)
# parser.add_argument('--log', help='Log to FILE in save folder; use - for stdout (default is log.txt)', metavar='FILE', default='log.txt')
# parser.add_argument('--no_timestamp_folder', help='Dont save to timestamp folder', action='store_true')
# parser.add_argument('--no_code_backup', help='Dont backup code', action='store_true')
# args = parser.parse_args()

if not args.no_timestamp_folder:
    time_string = datetime.now().strftime('%Y-%m-%d-%H-%M-%S')
    root_folder = os.path.join(args.save_folder, '%s_%s_%s_%d' % (args.model, args.setting, time_string, os.getpid()))
else:
    root_folder = args.save_folder
if not os.path.exists(root_folder):
    os.makedirs(root_folder)

if args.log != '-':
    sys.stdout = open(os.path.join(root_folder, args.log), 'w')


In [4]:
model = importlib.import_module(args.model)
setting_path = os.path.join("./", args.model)
sys.path.append(setting_path)
setting = importlib.import_module(args.setting)

num_epochs = args.epochs or setting.num_epochs
batch_size = args.batch_size or setting.batch_size
sample_num = setting.sample_num
step_val = setting.step_val
label_weights_list = setting.label_weights
rotation_range = setting.rotation_range
rotation_range_val = setting.rotation_range_val
scaling_range = setting.scaling_range
scaling_range_val = setting.scaling_range_val
jitter = setting.jitter
jitter_val = setting.jitter_val


In [5]:
# Prepare inputs
print('{}-Preparing datasets...'.format(datetime.now()))
is_list_of_h5_list = not data_utils.is_h5_list(args.filelist)
if is_list_of_h5_list:
    seg_list = data_utils.load_seg_list(args.filelist)
    seg_list_idx = 0
    filelist_train = seg_list[seg_list_idx]
    seg_list_idx = seg_list_idx + 1
else:
    filelist_train = args.filelist
data_train, _, data_num_train, label_train, _ = data_utils.load_seg(filelist_train)
data_val, _, data_num_val, label_val, _ = data_utils.load_seg(args.filelist_val)

# shuffle
data_train, data_num_train, label_train = \
    data_utils.grouped_shuffle([data_train, data_num_train, label_train])

num_train = data_train.shape[0]
point_num = data_train.shape[1]
num_val = data_val.shape[0]
print('{}-{:d}/{:d} training/validation samples.'.format(datetime.now(), num_train, num_val))
batch_num = (num_train * num_epochs + batch_size - 1) // batch_size
print('{}-{:d} training batches.'.format(datetime.now(), batch_num))
batch_num_val = math.ceil(num_val / batch_size)
print('{}-{:d} testing batches per test.'.format(datetime.now(), batch_num_val))



2019-02-10 21:10:01.714252-Preparing datasets...
2019-02-10 21:10:07.949720-1180/1232 training/validation samples.
2019-02-10 21:10:07.950069-25174 training batches.
2019-02-10 21:10:07.950145-103 testing batches per test.


In [6]:
setting.use_extra_features

False

In [7]:
######################################################################
# Placeholders
indices = tf.placeholder(tf.int32, shape=(None, None, 2), name="indices")
xforms = tf.placeholder(tf.float32, shape=(None, 3, 3), name="xforms")
rotations = tf.placeholder(tf.float32, shape=(None, 3, 3), name="rotations")
jitter_range = tf.placeholder(tf.float32, shape=(1), name="jitter_range")
global_step = tf.Variable(0, trainable=False, name='global_step')
is_training = tf.placeholder(tf.bool, name='is_training')

pts_fts = tf.placeholder(tf.float32, shape=(None, point_num, setting.data_dim), name='pts_fts')
labels_seg = tf.placeholder(tf.int64, shape=(None, point_num), name='labels_seg')
labels_weights = tf.placeholder(tf.float32, shape=(None, point_num), name='labels_weights')

######################################################################
pts_fts_sampled = tf.gather_nd(pts_fts, indices=indices, name='pts_fts_sampled')
features_augmented = None
if setting.data_dim > 3:
    points_sampled, features_sampled = tf.split(pts_fts_sampled,
                                                [3, setting.data_dim - 3],
                                                axis=-1,
                                                name='split_points_features')
    if setting.use_extra_features:
        if setting.with_normal_feature:
            if setting.data_dim < 6:
                print('Only 3D normals are supported!')
                exit()
            elif setting.data_dim == 6:
                features_augmented = pf.augment(features_sampled, rotations)
            else:
                normals, rest = tf.split(features_sampled, [3, setting.data_dim - 6])
                normals_augmented = pf.augment(normals, rotations)
                features_augmented = tf.concat([normals_augmented, rest], axis=-1)
        else:
            features_augmented = features_sampled
else:
    points_sampled = pts_fts_sampled
points_augmented = pf.augment(points_sampled, xforms, jitter_range)

labels_sampled = tf.gather_nd(labels_seg, indices=indices, name='labels_sampled')
labels_weights_sampled = tf.gather_nd(labels_weights, indices=indices, name='labels_weight_sampled')



In [8]:
labels_sampled

<tf.Tensor 'labels_sampled:0' shape=(?, ?) dtype=int64>

In [9]:
net = model.Net(points_augmented, features_augmented, is_training, setting)
logits = net.logits
probs = tf.nn.softmax(logits, name='probs')
predictions = tf.argmax(probs, axis=-1, name='predictions')

loss_op = tf.losses.sparse_softmax_cross_entropy(labels=labels_sampled, logits=logits,
                                                 weights=labels_weights_sampled)

with tf.name_scope('metrics'):
    loss_mean_op, loss_mean_update_op = tf.metrics.mean(loss_op)
    t_1_acc_op, t_1_acc_update_op = tf.metrics.accuracy(labels_sampled, predictions, weights=labels_weights_sampled)
    t_1_per_class_acc_op, t_1_per_class_acc_update_op = \
        tf.metrics.mean_per_class_accuracy(labels_sampled, predictions, setting.num_class,
                                           weights=labels_weights_sampled)
reset_metrics_op = tf.variables_initializer([var for var in tf.local_variables()
                                             if var.name.split('/')[0] == 'metrics'])


_ = tf.summary.scalar('loss/train', tensor=loss_mean_op, collections=['train'])
_ = tf.summary.scalar('t_1_acc/train', tensor=t_1_acc_op, collections=['train'])
_ = tf.summary.scalar('t_1_per_class_acc/train', tensor=t_1_per_class_acc_op, collections=['train'])

_ = tf.summary.scalar('loss/val', tensor=loss_mean_op, collections=['val'])
_ = tf.summary.scalar('t_1_acc/val', tensor=t_1_acc_op, collections=['val'])
_ = tf.summary.scalar('t_1_per_class_acc/val', tensor=t_1_per_class_acc_op, collections=['val'])

lr_exp_op = tf.train.exponential_decay(setting.learning_rate_base, global_step, setting.decay_steps,
                                       setting.decay_rate, staircase=True)
lr_clip_op = tf.maximum(lr_exp_op, setting.learning_rate_min)
_ = tf.summary.scalar('learning_rate', tensor=lr_clip_op, collections=['train'])
reg_loss = setting.weight_decay * tf.losses.get_regularization_loss()
if setting.optimizer == 'adam':
    optimizer = tf.train.AdamOptimizer(learning_rate=lr_clip_op, epsilon=setting.epsilon)
elif setting.optimizer == 'momentum':
    optimizer = tf.train.MomentumOptimizer(learning_rate=lr_clip_op, momentum=setting.momentum, use_nesterov=True)
update_ops = tf.get_collection(tf.GraphKeys.UPDATE_OPS)
with tf.control_dependencies(update_ops):
    train_op = optimizer.minimize(loss_op + reg_loss, global_step=global_step)

init_op = tf.group(tf.global_variables_initializer(), tf.local_variables_initializer())

saver = tf.train.Saver(max_to_keep=None)

folder_ckpt = os.path.join(root_folder, 'ckpts')
if not os.path.exists(folder_ckpt):
    os.makedirs(folder_ckpt)

folder_summary = os.path.join(root_folder, 'summary')
if not os.path.exists(folder_summary):
    os.makedirs(folder_summary)

parameter_num = np.sum([np.prod(v.shape.as_list()) for v in tf.trainable_variables()])
print('{}-Parameter number: {:d}.'.format(datetime.now(), parameter_num))


Instructions for updating:
keep_dims is deprecated, use keepdims instead
Instructions for updating:
Use the retry module or similar alternatives.
2019-02-10 21:10:17.335668-Parameter number: 3278203.


In [10]:
config = tf.ConfigProto()
config.gpu_options.allow_growth = True
config.allow_soft_placement = True

In [11]:

with tf.Session(config=config) as sess:
    summaries_op = tf.summary.merge_all('train')
    summaries_val_op = tf.summary.merge_all('val')
    summary_writer = tf.summary.FileWriter(folder_summary, sess.graph)

    sess.run(init_op)

    # Load the model
    if args.load_ckpt is not None:
        saver.restore(sess, args.load_ckpt)
        print('{}-Checkpoint loaded from {}!'.format(datetime.now(), args.load_ckpt))
    else:
        latest_ckpt = tf.train.latest_checkpoint(folder_ckpt)
        if latest_ckpt:
            print('{}-Found checkpoint {}'.format(datetime.now(), latest_ckpt))
            saver.restore(sess, latest_ckpt)
            print('{}-Checkpoint loaded from {} (Iter {})'.format(
                datetime.now(), latest_ckpt, sess.run(global_step)))

    for batch_idx_train in range(batch_num):
        ######################################################################
        # Validation
        if (batch_idx_train % step_val == 0 and (batch_idx_train != 0 or args.load_ckpt is not None)) \
                or batch_idx_train == batch_num - 1:
            filename_ckpt = os.path.join(folder_ckpt, 'iter')
            saver.save(sess, filename_ckpt, global_step=global_step)
            print('{}-Checkpoint saved to {}!'.format(datetime.now(), filename_ckpt))

            sess.run(reset_metrics_op)
            for batch_val_idx in range(batch_num_val):
                start_idx = batch_size * batch_val_idx
                end_idx = min(start_idx + batch_size, num_val)
                batch_size_val = end_idx - start_idx
                points_batch = data_val[start_idx:end_idx, ...]
                points_num_batch = data_num_val[start_idx:end_idx, ...]
                labels_batch = label_val[start_idx:end_idx, ...]
                weights_batch = np.array(label_weights_list)[labels_batch]

                xforms_np, rotations_np = pf.get_xforms(batch_size_val,
                                                        rotation_range=rotation_range_val,
                                                        scaling_range=scaling_range_val,
                                                        order=setting.rotation_order)
                sess.run([loss_mean_update_op, t_1_acc_update_op, t_1_per_class_acc_update_op],
                         feed_dict={
                             pts_fts: points_batch,
                             indices: pf.get_indices(batch_size_val, sample_num, points_num_batch),
                             xforms: xforms_np,
                             rotations: rotations_np,
                             jitter_range: np.array([jitter_val]),
                             labels_seg: labels_batch,
                             labels_weights: weights_batch,
                             is_training: False,
                         })
            loss_val, t_1_acc_val, t_1_per_class_acc_val, summaries_val, step = sess.run(
                [loss_mean_op, t_1_acc_op, t_1_per_class_acc_op, summaries_val_op, global_step])
            summary_writer.add_summary(summaries_val, step)
            print('{}-[Val  ]-Average:      Loss: {:.4f}  T-1 Acc: {:.4f}  T-1 mAcc: {:.4f}'
                  .format(datetime.now(), loss_val, t_1_acc_val, t_1_per_class_acc_val))
            sys.stdout.flush()
        ######################################################################

        ######################################################################
        # Training
        start_idx = (batch_size * batch_idx_train) % num_train
        end_idx = min(start_idx + batch_size, num_train)
        batch_size_train = end_idx - start_idx
        points_batch = data_train[start_idx:end_idx, ...]
        points_num_batch = data_num_train[start_idx:end_idx, ...]
        labels_batch = label_train[start_idx:end_idx, ...]
        weights_batch = np.array(label_weights_list)[labels_batch]

        if start_idx + batch_size_train == num_train:
            if is_list_of_h5_list:
                filelist_train_prev = seg_list[(seg_list_idx - 1) % len(seg_list)]
                filelist_train = seg_list[seg_list_idx % len(seg_list)]
                if filelist_train != filelist_train_prev:
                    data_train, _, data_num_train, label_train, _ = data_utils.load_seg(filelist_train)
                    num_train = data_train.shape[0]
                seg_list_idx = seg_list_idx + 1
            data_train, data_num_train, label_train = \
                data_utils.grouped_shuffle([data_train, data_num_train, label_train])

        offset = int(random.gauss(0, sample_num * setting.sample_num_variance))
        offset = max(offset, -sample_num * setting.sample_num_clip)
        offset = min(offset, sample_num * setting.sample_num_clip)
        sample_num_train = sample_num + offset
        xforms_np, rotations_np = pf.get_xforms(batch_size_train,
                                                rotation_range=rotation_range,
                                                scaling_range=scaling_range,
                                                order=setting.rotation_order)
        sess.run(reset_metrics_op)
        sess.run([train_op, loss_mean_update_op, t_1_acc_update_op, t_1_per_class_acc_update_op],
                 feed_dict={
                     pts_fts: points_batch,
                     indices: pf.get_indices(batch_size_train, sample_num_train, points_num_batch),
                     xforms: xforms_np,
                     rotations: rotations_np,
                     jitter_range: np.array([jitter]),
                     labels_seg: labels_batch,
                     labels_weights: weights_batch,
                     is_training: True,
                 })
        if batch_idx_train % 10 == 0:
            loss, t_1_acc, t_1_per_class_acc, summaries, step = sess.run([loss_mean_op,
                                                                    t_1_acc_op,
                                                                    t_1_per_class_acc_op,
                                                                    summaries_op,
                                                                    global_step])
            summary_writer.add_summary(summaries, step)
            print('{}-[Train]-Iter: {:06d}  Loss: {:.4f}  T-1 Acc: {:.4f}  T-1 mAcc: {:.4f}'
                  .format(datetime.now(), step, loss, t_1_acc, t_1_per_class_acc))
            sys.stdout.flush()
        ######################################################################
    print('{}-Done!'.format(datetime.now()))


INFO:tensorflow:Restoring parameters from ../models/seg/pointcnn_seg_amsterdam_x4_12288_fps_2019-02-05-22-40-26_20662/ckpts/iter-25173
2019-02-10 21:10:22.322163-Checkpoint loaded from ../models/seg/pointcnn_seg_amsterdam_x4_12288_fps_2019-02-05-22-40-26_20662/ckpts/iter-25173!
INFO:tensorflow:../models/seg/pointcnn_seg_amsterdam_x4_12288_fps_2019-02-10-21-10-01_17799/ckpts/iter-25173 is not in all_model_checkpoint_paths. Manually adding it.
2019-02-10 21:10:23.691484-Checkpoint saved to ../models/seg/pointcnn_seg_amsterdam_x4_12288_fps_2019-02-10-21-10-01_17799/ckpts/iter!
2019-02-10 21:11:17.654903-[Val  ]-Average:      Loss: 0.0813  T-1 Acc: 0.9715  T-1 mAcc: 0.1764
2019-02-10 21:11:26.291894-[Train]-Iter: 025174  Loss: 0.2194  T-1 Acc: 0.9346  T-1 mAcc: 0.1312
2019-02-10 21:11:34.419462-[Train]-Iter: 025184  Loss: 0.0938  T-1 Acc: 0.9701  T-1 mAcc: 0.1462
2019-02-10 21:11:42.571254-[Train]-Iter: 025194  Loss: 0.1314  T-1 Acc: 0.9389  T-1 mAcc: 0.1541
2019-02-10 21:11:50.761134-[Tra

2019-02-10 21:22:35.236715-[Train]-Iter: 025914  Loss: 0.0777  T-1 Acc: 0.9707  T-1 mAcc: 0.1070
2019-02-10 21:22:43.353185-[Train]-Iter: 025924  Loss: 0.0429  T-1 Acc: 0.9855  T-1 mAcc: 0.1216
2019-02-10 21:22:51.500056-[Train]-Iter: 025934  Loss: 0.1080  T-1 Acc: 0.9609  T-1 mAcc: 0.1339
2019-02-10 21:22:59.673693-[Train]-Iter: 025944  Loss: 0.0573  T-1 Acc: 0.9800  T-1 mAcc: 0.1181
2019-02-10 21:23:07.805389-[Train]-Iter: 025954  Loss: 0.0990  T-1 Acc: 0.9666  T-1 mAcc: 0.1461
2019-02-10 21:23:15.928026-[Train]-Iter: 025964  Loss: 0.0618  T-1 Acc: 0.9800  T-1 mAcc: 0.1147
2019-02-10 21:23:24.111602-[Train]-Iter: 025974  Loss: 0.0659  T-1 Acc: 0.9781  T-1 mAcc: 0.1499
2019-02-10 21:23:36.138289-[Train]-Iter: 025984  Loss: 0.0852  T-1 Acc: 0.9723  T-1 mAcc: 0.1125
2019-02-10 21:23:44.309249-[Train]-Iter: 025994  Loss: 0.0553  T-1 Acc: 0.9828  T-1 mAcc: 0.1374
2019-02-10 21:23:52.452245-[Train]-Iter: 026004  Loss: 0.3477  T-1 Acc: 0.8959  T-1 mAcc: 0.0993
2019-02-10 21:24:00.594846-[Tr

2019-02-10 21:35:05.412859-[Train]-Iter: 026684  Loss: 0.0580  T-1 Acc: 0.9788  T-1 mAcc: 0.1116
2019-02-10 21:35:13.546114-[Train]-Iter: 026694  Loss: 0.0813  T-1 Acc: 0.9726  T-1 mAcc: 0.1450
2019-02-10 21:35:21.685245-[Train]-Iter: 026704  Loss: 0.0906  T-1 Acc: 0.9715  T-1 mAcc: 0.1468
2019-02-10 21:35:32.837880-[Train]-Iter: 026714  Loss: 0.0520  T-1 Acc: 0.9845  T-1 mAcc: 0.1460
2019-02-10 21:35:40.874651-[Train]-Iter: 026724  Loss: 0.1223  T-1 Acc: 0.9548  T-1 mAcc: 0.1752
2019-02-10 21:35:48.865847-[Train]-Iter: 026734  Loss: 0.0959  T-1 Acc: 0.9665  T-1 mAcc: 0.1707
2019-02-10 21:35:56.926561-[Train]-Iter: 026744  Loss: 0.0770  T-1 Acc: 0.9736  T-1 mAcc: 0.1740
2019-02-10 21:36:04.969175-[Train]-Iter: 026754  Loss: 0.1169  T-1 Acc: 0.9590  T-1 mAcc: 0.1182
2019-02-10 21:36:13.041867-[Train]-Iter: 026764  Loss: 0.0628  T-1 Acc: 0.9775  T-1 mAcc: 0.1298
2019-02-10 21:36:21.149781-[Train]-Iter: 026774  Loss: 0.1650  T-1 Acc: 0.9535  T-1 mAcc: 0.1462
2019-02-10 21:36:29.278087-[Tr

2019-02-10 21:47:14.591551-[Train]-Iter: 027494  Loss: 0.0485  T-1 Acc: 0.9824  T-1 mAcc: 0.1201
2019-02-10 21:47:22.706315-[Train]-Iter: 027504  Loss: 0.0605  T-1 Acc: 0.9778  T-1 mAcc: 0.1328
2019-02-10 21:47:30.817837-[Train]-Iter: 027514  Loss: 0.0712  T-1 Acc: 0.9725  T-1 mAcc: 0.1459
2019-02-10 21:47:38.923109-[Train]-Iter: 027524  Loss: 0.0519  T-1 Acc: 0.9822  T-1 mAcc: 0.1367
2019-02-10 21:47:47.014366-[Train]-Iter: 027534  Loss: 0.0402  T-1 Acc: 0.9863  T-1 mAcc: 0.1356
2019-02-10 21:47:58.289195-[Train]-Iter: 027544  Loss: 0.1561  T-1 Acc: 0.9532  T-1 mAcc: 0.1572
2019-02-10 21:48:06.356507-[Train]-Iter: 027554  Loss: 0.0791  T-1 Acc: 0.9717  T-1 mAcc: 0.1426
2019-02-10 21:48:14.468747-[Train]-Iter: 027564  Loss: 0.1104  T-1 Acc: 0.9605  T-1 mAcc: 0.1734
2019-02-10 21:48:22.944982-[Train]-Iter: 027574  Loss: 0.0551  T-1 Acc: 0.9822  T-1 mAcc: 0.1459
2019-02-10 21:48:30.984742-[Train]-Iter: 027584  Loss: 0.0961  T-1 Acc: 0.9635  T-1 mAcc: 0.1096
2019-02-10 21:48:39.050283-[Tr

2019-02-10 21:59:39.101990-[Train]-Iter: 028264  Loss: 0.0828  T-1 Acc: 0.9706  T-1 mAcc: 0.1254
2019-02-10 21:59:50.358773-[Train]-Iter: 028274  Loss: 0.1422  T-1 Acc: 0.9474  T-1 mAcc: 0.1358
2019-02-10 21:59:58.406305-[Train]-Iter: 028284  Loss: 0.0970  T-1 Acc: 0.9691  T-1 mAcc: 0.1411
2019-02-10 22:00:06.492755-[Train]-Iter: 028294  Loss: 0.1180  T-1 Acc: 0.9534  T-1 mAcc: 0.1335
2019-02-10 22:00:14.598401-[Train]-Iter: 028304  Loss: 0.1140  T-1 Acc: 0.9593  T-1 mAcc: 0.1118
2019-02-10 22:00:22.671479-[Train]-Iter: 028314  Loss: 0.1072  T-1 Acc: 0.9599  T-1 mAcc: 0.1374
2019-02-10 22:00:30.752475-[Train]-Iter: 028324  Loss: 0.1424  T-1 Acc: 0.9489  T-1 mAcc: 0.1485
2019-02-10 22:00:38.824838-[Train]-Iter: 028334  Loss: 0.0906  T-1 Acc: 0.9734  T-1 mAcc: 0.1430
2019-02-10 22:00:46.888073-[Train]-Iter: 028344  Loss: 0.0842  T-1 Acc: 0.9701  T-1 mAcc: 0.1578
2019-02-10 22:00:54.924063-[Train]-Iter: 028354  Loss: 0.0720  T-1 Acc: 0.9751  T-1 mAcc: 0.1587
2019-02-10 22:01:03.177628-[Tr

...

2019-02-11 02:53:48.460046-Checkpoint saved to ../models/seg/pointcnn_seg_amsterdam_x4_12288_fps_2019-02-10-21-10-01_17799/ckpts/iter!
2019-02-11 02:54:39.779518-[Val  ]-Average:      Loss: 0.0745  T-1 Acc: 0.9731  T-1 mAcc: 0.1745
2019-02-11 02:54:40.595730-[Train]-Iter: 047174  Loss: 0.1007  T-1 Acc: 0.9702  T-1 mAcc: 0.1405
2019-02-11 02:54:48.614349-[Train]-Iter: 047184  Loss: 0.1169  T-1 Acc: 0.9581  T-1 mAcc: 0.1548
2019-02-11 02:54:56.666407-[Train]-Iter: 047194  Loss: 0.0928  T-1 Acc: 0.9721  T-1 mAcc: 0.1439
2019-02-11 02:55:04.717072-[Train]-Iter: 047204  Loss: 0.0997  T-1 Acc: 0.9620  T-1 mAcc: 0.1439
2019-02-11 02:55:12.829961-[Train]-Iter: 047214  Loss: 0.1002  T-1 Acc: 0.9654  T-1 mAcc: 0.1139
2019-02-11 02:55:20.917147-[Train]-Iter: 047224  Loss: 0.1261  T-1 Acc: 0.9549  T-1 mAcc: 0.1632
2019-02-11 02:55:28.999175-[Train]-Iter: 047234  Loss: 0.0989  T-1 Acc: 0.9677  T-1 mAcc: 0.1471
2019-02-11 02:55:37.070566-[Train]-Iter: 047244  Loss: 0.0867  T-1 Acc: 0.9693  T-1 mAcc:

2019-02-11 03:06:18.725518-[Train]-Iter: 047954  Loss: 0.1729  T-1 Acc: 0.9472  T-1 mAcc: 0.1545
2019-02-11 03:06:26.851307-[Train]-Iter: 047964  Loss: 0.0945  T-1 Acc: 0.9636  T-1 mAcc: 0.1067
2019-02-11 03:06:34.941646-[Train]-Iter: 047974  Loss: 0.0936  T-1 Acc: 0.9660  T-1 mAcc: 0.1737
2019-02-11 03:06:43.010756-[Train]-Iter: 047984  Loss: 0.1662  T-1 Acc: 0.9420  T-1 mAcc: 0.1404
2019-02-11 03:06:51.093270-[Train]-Iter: 047994  Loss: 0.2258  T-1 Acc: 0.9256  T-1 mAcc: 0.1501
2019-02-11 03:07:02.162579-[Train]-Iter: 048004  Loss: 0.0553  T-1 Acc: 0.9795  T-1 mAcc: 0.1452
2019-02-11 03:07:10.216712-[Train]-Iter: 048014  Loss: 0.0670  T-1 Acc: 0.9805  T-1 mAcc: 0.1312
2019-02-11 03:07:18.266872-[Train]-Iter: 048024  Loss: 0.0526  T-1 Acc: 0.9837  T-1 mAcc: 0.1091
2019-02-11 03:07:26.354349-[Train]-Iter: 048034  Loss: 0.1089  T-1 Acc: 0.9622  T-1 mAcc: 0.1437
2019-02-11 03:07:34.437084-[Train]-Iter: 048044  Loss: 0.1475  T-1 Acc: 0.9509  T-1 mAcc: 0.1281
2019-02-11 03:07:42.556584-[Tr

2019-02-11 03:18:44.445038-[Train]-Iter: 048724  Loss: 0.0573  T-1 Acc: 0.9831  T-1 mAcc: 0.1400
2019-02-11 03:18:52.518499-[Train]-Iter: 048734  Loss: 0.0737  T-1 Acc: 0.9770  T-1 mAcc: 0.1451
2019-02-11 03:19:00.627404-[Train]-Iter: 048744  Loss: 0.0755  T-1 Acc: 0.9753  T-1 mAcc: 0.1347
2019-02-11 03:19:08.753185-[Train]-Iter: 048754  Loss: 0.0754  T-1 Acc: 0.9745  T-1 mAcc: 0.1438
2019-02-11 03:19:16.877299-[Train]-Iter: 048764  Loss: 0.1048  T-1 Acc: 0.9607  T-1 mAcc: 0.1160
2019-02-11 03:19:24.983195-[Train]-Iter: 048774  Loss: 0.0853  T-1 Acc: 0.9707  T-1 mAcc: 0.1437
2019-02-11 03:19:34.949115-[Train]-Iter: 048784  Loss: 0.1069  T-1 Acc: 0.9641  T-1 mAcc: 0.1432
2019-02-11 03:19:42.984369-[Train]-Iter: 048794  Loss: 0.0703  T-1 Acc: 0.9750  T-1 mAcc: 0.1232
2019-02-11 03:19:50.832446-[Train]-Iter: 048804  Loss: 0.0657  T-1 Acc: 0.9767  T-1 mAcc: 0.1236
2019-02-11 03:19:58.899273-[Train]-Iter: 048814  Loss: 0.0827  T-1 Acc: 0.9709  T-1 mAcc: 0.1420
2019-02-11 03:20:06.979276-[Tr

2019-02-11 03:30:49.448906-[Train]-Iter: 049534  Loss: 0.0432  T-1 Acc: 0.9866  T-1 mAcc: 0.1465
2019-02-11 03:30:57.567934-[Train]-Iter: 049544  Loss: 0.0888  T-1 Acc: 0.9710  T-1 mAcc: 0.1546
2019-02-11 03:31:05.631035-[Train]-Iter: 049554  Loss: 0.0459  T-1 Acc: 0.9847  T-1 mAcc: 0.1473
2019-02-11 03:31:16.434355-[Train]-Iter: 049564  Loss: 0.0865  T-1 Acc: 0.9731  T-1 mAcc: 0.1117
2019-02-11 03:31:24.489088-[Train]-Iter: 049574  Loss: 0.0896  T-1 Acc: 0.9696  T-1 mAcc: 0.1463
2019-02-11 03:31:32.543056-[Train]-Iter: 049584  Loss: 0.0748  T-1 Acc: 0.9755  T-1 mAcc: 0.1467
2019-02-11 03:31:40.632255-[Train]-Iter: 049594  Loss: 0.0638  T-1 Acc: 0.9780  T-1 mAcc: 0.1656
2019-02-11 03:31:48.701905-[Train]-Iter: 049604  Loss: 0.0791  T-1 Acc: 0.9721  T-1 mAcc: 0.1118
2019-02-11 03:31:56.794365-[Train]-Iter: 049614  Loss: 0.0728  T-1 Acc: 0.9765  T-1 mAcc: 0.1446
2019-02-11 03:32:04.914950-[Train]-Iter: 049624  Loss: 0.0624  T-1 Acc: 0.9782  T-1 mAcc: 0.1231
2019-02-11 03:32:13.008835-[Tr

2019-02-11 03:43:10.819507-[Train]-Iter: 050304  Loss: 0.0586  T-1 Acc: 0.9830  T-1 mAcc: 0.1607
2019-02-11 03:43:21.708762-[Train]-Iter: 050314  Loss: 0.0852  T-1 Acc: 0.9704  T-1 mAcc: 0.1634
2019-02-11 03:43:29.745325-[Train]-Iter: 050324  Loss: 0.0990  T-1 Acc: 0.9683  T-1 mAcc: 0.1509
2019-02-11 03:43:37.820954-[Train]-Iter: 050334  Loss: 0.0521  T-1 Acc: 0.9822  T-1 mAcc: 0.1674
2019-02-11 03:43:45.900520-[Train]-Iter: 050344  Loss: 0.0847  T-1 Acc: 0.9690  T-1 mAcc: 0.1769
INFO:tensorflow:../models/seg/pointcnn_seg_amsterdam_x4_12288_fps_2019-02-10-21-10-01_17799/ckpts/iter-50346 is not in all_model_checkpoint_paths. Manually adding it.
2019-02-11 03:43:48.443455-Checkpoint saved to ../models/seg/pointcnn_seg_amsterdam_x4_12288_fps_2019-02-10-21-10-01_17799/ckpts/iter!
2019-02-11 03:44:39.857156-[Val  ]-Average:      Loss: 0.0971  T-1 Acc: 0.9682  T-1 mAcc: 0.1742
2019-02-11 03:44:40.673820-Done!
